In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

Зададим простейшие одномерные данные:

In [ ]:
data = np.array([1, -2, -3, -1, 2, 3])[:, np.newaxis]
target = np.array([0, 0, 0, 1, 1, 1])

In [ ]:
plt.scatter(data.ravel(), target)

Реализуем вычисление функционала качества:

In [ ]:
def sigmoid(x):
    ### your code here
    
def qual(w, X, y, C=0):
    ### your code here
    ### 0, 1 -> -1, +1
    

In [ ]:
plt.scatter(data.ravel(), target)
ls = np.linspace(-3, 3, 100)
for w in [0.1, 1, 10]:
    print(w, qual([w], data, target))
    plt.plot(ls, sigmoid(w*ls), label=str(w))
plt.legend()

Не выгодно подгоняться под "правильные" ответы, лучше всего что-то среднее.

In [ ]:
from scipy.optimize import minimize

In [ ]:
def grad(w, X, y, C=0):
    ### your code here
    ### 0, 1 -> -1, +1

class Callback:
    def __init__(self):
        self.ws = []
        
    def call(self, w):
        self.ws.append(np.copy(w))

class MyLR:
    def __init__(self, C=0, lr=1):
        self.C = C
        self.lr = lr
    
    def fit(self, X, y):
        d = X.shape[1] 
        w = np.zeros(d) #  инициализация весов
        self.callback = Callback()
        # обучение весов
        self.res = minimize(qual, w,  
                    method="L-BFGS-B",
                    jac=grad,
                    args=(X, y, self.C),
                    callback=self.callback.call)
        self.w = self.res.x
    
    def predict(self, X):
        ### your code here
    
    def predict_proba(self, X):
        ### your code here

Сравним результат нашей реализации и sklearn:

In [ ]:
from sklearn.linear_model import LogisticRegression as LR

In [ ]:
lr = LR(C=10**8) # "no reg"
lr.fit(data, target)
lr.coef_

In [ ]:
mylr = MyLR()
mylr.fit(data, target)
mylr.w

Процесс обучения весов:

In [ ]:
mylr.callback.ws

In [ ]:
plt.plot(mylr.callback.ws)

Теперь посмотрим, что будет, если данные линейно разделимы:

In [ ]:
data2 = np.array([-1, -2, -3, 1, 2, 3])[:, np.newaxis]
target = np.array([0, 0, 0, 1, 1, 1])

In [ ]:
plt.scatter(data2.ravel(), target)

In [ ]:
mylr = MyLR()
mylr.fit(data2, target)
mylr.w

In [ ]:
plt.plot(mylr.callback.ws)

In [ ]:
plt.scatter(data2.ravel(), target)
plt.plot(ls, sigmoid(ls*mylr.w))

Попробуем добавить регуляризацию:

In [ ]:
plt.scatter(data2.ravel(), target)
for C in [0, 10, 100, 10**9]:
    mylr = MyLR(C=C)
    mylr.fit(data2, target)
    points = np.linspace(-3.5, 3.5, 1000)
    preds = mylr.predict_proba(points[:, np.newaxis])
    plt.plot(points, preds, label=str(C))
plt.legend()

То же на data:

In [ ]:
plt.scatter(data.ravel(), target)
for C in [0, 10, 100, 10**9]:
    mylr = MyLR(C=C)
    mylr.fit(data, target)
    points = np.linspace(-3.5, 3.5, 1000)
    preds = mylr.predict_proba(points[:, np.newaxis])
    plt.plot(points, preds, label=str(C))
plt.legend()